In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from esios import *

Create dictionary with Indicator Name and Number of Indicator

In [2]:
indicatorsDict = {
#                   'Previsión diaria de la demanda eléctrica peninsular' : 460,
#                   /'Precio medio horario componente mercado diario' : 805,
#     'demand': 460,
#     'price': 805,
#     'nuclear': 1153,
#     'hydro_uhg':1150,
#     'hydro_nouhg':1151,
    'wind':1159,
#     'wind_m':1160,
#     'thermo':1162,
#     'photo':1161,
#     'bombeo':1152
#                   'Generación T.Real nuclear' : 549,
#                   'Generación T.Real hidráulica' : 546,
#                   'Generación T.Real eólica' : 551,
#                   'Generación T.Real Solar térmica' : 1294,
#                   'Generación T.Real Solar fotovoltaica' : 1295
                 }

indicatorsItems = indicatorsDict.items()   # To iterate over name and number at the same time

Download indicators data from 01/01/2016 to 01/06/2020 and save it into `".csv"` files.

In [4]:
start_date = "2018-01-01"
end_date = "2020-06-01"
start_ = start_date + 'T00:00:00'
end_ = end_date + 'T23:50:00'
token = '6cc21e0b60e9931e7522a6ce72a1a09f3a6fadc6f08b142f956db142c6858bc2'    # Introduce ESIOS token
esios = ESIOS(token)
country = 'Spain' #Spain, France or Portugal are the options

for indicatorName, indicatorValue in indicatorsItems:
    print ('Start Date: ' + start_date)
    print ('End Date: ' + end_date)
    indicators_ = list()
    indicators_.append(indicatorValue)
    dfmul , df_list, names = esios.get_multiple_series(indicators_, start_, end_, country)
    df = dfmul[names]
    df = df.reset_index()
    df.columns = ['date', indicatorName]
    df['date'] = df['date'].str.replace('.', ' ')
    df['date'] = df['date'].str.split().str[0]
    df['date'] = df['date'].str.replace('T', ' ')
    # Export to .csv file
    try:
        os.stat("Files/")
    except:
        os.mkdir("Files/")
    df.to_csv(path_or_buf= 'Files/' + str(indicatorValue) + '.csv', sep='^', index=False)
    print('Generated:' + str(indicatorValue))

Analyzing indicators...
Start Date: 2018-01-01
End Date: 2020-06-01


HTTPError: HTTP Error 502: Proxy Error

Read `'.csv'` files generated in different DataFrames.

In [ ]:
with open('Files/460.csv', 'r') as fichero:     
    dfDemand = pd.read_csv(fichero, sep = '^', dtype='object')
    dfDemand['date']=pd.to_datetime(dfDemand['date'])
    
with open('Files/1294.csv', 'r') as fichero:     
    dfSolarT = pd.read_csv(fichero, sep = '^', dtype='object')
    dfSolarT['date']=pd.to_datetime(dfSolarT['date'])
    
with open('Files/1295.csv', 'r') as fichero:     
    dfSolar = pd.read_csv(fichero, sep = '^', dtype='object')
    dfSolar['date']=pd.to_datetime(dfSolar['date']) 
    
with open('Files/551.csv', 'r') as fichero:     
    dfEolic = pd.read_csv(fichero, sep = '^', dtype='object')
    dfEolic['date']=pd.to_datetime(dfEolic['date'])
    
with open('Files/805.csv', 'r') as fichero:     
    dfPrice = pd.read_csv(fichero, sep = '^', dtype='object')
    dfPrice['date']=pd.to_datetime(dfPrice['date']) 
        
with open('Files/546.csv', 'r') as fichero:     
    dfHidro = pd.read_csv(fichero, sep = '^', dtype='object')
    dfHidro['date']=pd.to_datetime(dfHidro['date']) 
        
with open('Files/549.csv', 'r') as fichero:     
    dfNuclear = pd.read_csv(fichero, sep = '^', dtype='object')
    dfNuclear['date']=pd.to_datetime(dfNuclear['date']) 

In [ ]:
dfDemand.sample(5)

In [ ]:
dfSolar.sample(5)

In [ ]:
dfEolic.sample(5)

In [ ]:
dfPrice.head(20)

In [ ]:
dfNuclear.sample(5)

In [ ]:
dfPrice=dfPrice.merge(dfDemand,left_on='date',right_on='date',how='inner')
dfPrice=dfPrice.merge(dfSolarT,left_on='date',right_on='date',how='inner')
dfPrice=dfPrice.merge(dfSolar,left_on='date',right_on='date',how='inner')
dfPrice=dfPrice.merge(dfEolic,left_on='date',right_on='date',how='inner')
dfPrice=dfPrice.merge(dfHidro,left_on='date',right_on='date',how='inner')
dfPrice=dfPrice.merge(dfNuclear,left_on='date',right_on='date',how='inner')

In [ ]:
dfPrice.dtypes

In [ ]:
dfPrice['price']=dfPrice['price'].astype(float)
dfPrice['demand']=dfPrice['demand'].astype(float)
dfPrice['hydro']=dfPrice['hydro'].astype(float)
dfPrice['wind']=dfPrice['wind'].astype(float)
dfPrice['photo']=dfPrice['photo'].astype(float)
dfPrice['nuclear']=dfPrice['nuclear'].astype(float)
dfPrice['thermo']=dfPrice['thermo'].astype(float)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression


In [ ]:
new_data = dfPrice[['wind','demand','photo','hydro','thermo','nuclear']]

In [ ]:
X = new_data.values
y = dfPrice['price'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2)

In [ ]:
#Fitting XGB regressor 
model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.4, gamma=0.3, subsample=0.9,
                               colsample_bytree=1, max_depth=5,objective='reg:squarederror')
model.fit(X_train,y_train)
print (model)

In [ ]:
output = model.predict(data=X_test)
output

In [ ]:
comp=pd.DataFrame({'pred':output,'real':y_test})
comp.head()

In [ ]:
comp.dtypes

In [ ]:
from scipy import stats
def r2(x, y):
    return stats.pearsonr(x, y)
def npr2(x,y):
    return np.corrcoef(x,y)[0,1]**2

In [ ]:
Xri = comp["pred"].to_numpy()
Yri = comp["real"].to_numpy()
Xr = np.reshape(Xri,(-1,1))
Yr = np.reshape(Yri,(-1,1))

In [ ]:
sns.set(style="darkgrid")
g = sns.jointplot("pred", "real", data=comp, kind="reg", color="grey", 
                  height=7,stat_func=npr2, scatter_kws={'s':1},
                  joint_kws={'color':'grey','line_kws':{'color':'r'}})

In [ ]:
pintar=dfPrice[dfPrice['date']>='2018-05-01 00:00:00']
pintar.head()

In [ ]:
new_pintar = pintar[['wind','demand','photo','nuclear','thermo','hydro']]

In [ ]:
y = pintar['price'].values
X = new_pintar.values

In [ ]:
output_2 = model.predict(data=X)
output_2

In [ ]:
comp2=pd.DataFrame({'predicted':output_2,'real':y})
comp2.head()

In [ ]:
gr = sns.jointplot("predicted", "real", data=comp2, kind="reg",
                  xlim=(0.2, 10), ylim=(0.2, 10), color="grey", 
                 height=5,stat_func=npr2, scatter_kws={'s':4},
                  joint_kws={'color':'grey','line_kws':{'color':'red'}}, ci=95)

In [ ]:
PRED=comp.rename(columns={'real':'price'})
PRED.head()

In [ ]:
pre=PRED.merge(dfPrice,left_on= 'price',right_on='price',how='inner')
pre.head()

In [ ]:
pre=pre.drop(['price'],axis=1)

In [ ]:
pre=pre.rename(columns={'pred':'price'})
pre.head()

In [ ]:
pre.head()

In [ ]:
dfPrice.head()

In [ ]:
dfPrice=dfPrice[['date','price']]
pre=pre[['date','price']]

In [ ]:
preds =pre.sort_values('date').drop_duplicates(subset='date',keep='last')
realdata =dfPrice.sort_values('date').drop_duplicates(subset=['date', 'price'])

In [ ]:
preds=preds.tail(2000)
realdata=realdata.tail(2000)

In [ ]:
import altair as alt
alt.Chart(preds).mark_line().encode(
    x="date:T",
    y="price"
).properties(
   width=800
)

In [ ]:
alt.Chart(realdata).mark_line().encode(
    x="date:T",
    y="price"
).properties(
   width=800
)